In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.datasets import fetch_20newsgroups
cats = ['rec.autos', 'rec.motorcycles',
        'sci.crypt', 'sci.electronics', 
        'sci.med', 'sci.space',
        'talk.politics.guns', 'talk.religion.misc',
        'rec.sport.baseball', 'rec.sport.hockey']
#cats = None
df = fetch_20newsgroups(subset = 'all', categories=cats)

from pprint import pprint
pprint(list(df.target_names))

['rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'talk.politics.guns',
 'talk.religion.misc']


In [3]:
df.filenames.shape

(9469,)

In [4]:
df.target.shape

(9469,)

In [5]:
# Convert to list
data = df.data

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<input>:5: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-5-1eed9f39b362>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-5-1eed9f39b362>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['From: (Brian C. Kensing) Subject: Re: Manual Shift Bigots Nntp-Posting-Host: '
 'cary112.its.rpi.edu Reply-To: Organization: Rensselaer Polytechnic '
 'Institute, Troy, NY. Lines: 10 I prefer a manual to an automatic as it '
 'should be. I believe that automatics should only be manufactured for people '
 'with physical disabilities who otherwise would not be able to drive. '
 'Automatic transmissions allow drivers to be lazy. More time is available to '
 'fiddle with the radio or to look at the scenery instead of concentrating on '
 'the road. The manual transmission keeps the drive always doing something, '
 'granted it isnt a large movement. Plus, driving should be FUN! Driving a '
 'manual is fun, driving an automatic is a chore. In the case of shift speed, '
 'automatics can be made to shift far faster that any human could move a '
 'stick. If I was racing, Id want and automatic. For normal driving go with '
 'the manual. ']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'brian', 'kensing', 'subject', 're', 'manual', 'shift', 'bigots', 'nntp', 'posting', 'host', 'cary', 'its', 'rpi', 'edu', 'reply', 'to', 'organization', 'rensselaer', 'polytechnic', 'institute', 'troy', 'ny', 'lines', 'prefer', 'manual', 'to', 'an', 'automatic', 'as', 'it', 'should', 'be', 'believe', 'that', 'automatics', 'should', 'only', 'be', 'manufactured', 'for', 'people', 'with', 'physical', 'disabilities', 'who', 'otherwise', 'would', 'not', 'be', 'able', 'to', 'drive', 'automatic', 'transmissions', 'allow', 'drivers', 'to', 'be', 'lazy', 'more', 'time', 'is', 'available', 'to', 'fiddle', 'with', 'the', 'radio', 'or', 'to', 'look', 'at', 'the', 'scenery', 'instead', 'of', 'concentrating', 'on', 'the', 'road', 'the', 'manual', 'transmission', 'keeps', 'the', 'drive', 'always', 'doing', 'something', 'granted', 'it', 'isnt', 'large', 'movement', 'plus', 'driving', 'should', 'be', 'fun', 'driving', 'manual', 'is', 'fun', 'driving', 'an', 'automatic', 'is', 'chore', 'in', '

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'brian', 'kensing', 'subject', 're', 'manual_shift_bigots', 'nntp_posting_host', 'cary', 'its', 'rpi_edu', 'reply', 'to', 'organization', 'rensselaer_polytechnic_institute_troy', 'ny', 'lines', 'prefer', 'manual', 'to', 'an', 'automatic', 'as', 'it', 'should', 'be', 'believe', 'that', 'automatics', 'should', 'only', 'be', 'manufactured', 'for', 'people', 'with', 'physical', 'disabilities', 'who', 'otherwise', 'would', 'not', 'be', 'able', 'to', 'drive', 'automatic_transmissions', 'allow', 'drivers', 'to', 'be', 'lazy', 'more', 'time', 'is', 'available', 'to', 'fiddle', 'with', 'the', 'radio', 'or', 'to', 'look', 'at', 'the', 'scenery', 'instead', 'of', 'concentrating', 'on', 'the', 'road', 'the', 'manual_transmission', 'keeps', 'the', 'drive', 'always', 'doing', 'something', 'granted', 'it', 'isnt', 'large', 'movement', 'plus', 'driving', 'should', 'be', 'fun', 'driving', 'manual', 'is', 'fun', 'driving', 'an', 'automatic', 'is', 'chore', 'in', 'the', 'case', 'of', 'shift', 's

In [8]:
# NLTK Stop words
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asergeev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# !python -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['brian', 'kens', 'manual_shift', 'bigot', 'nntp_post', 'host', 'cary', 'rpi', 'reply', 'organization', 'rensselaer_polytechnic', 'institute_troy', 'line', 'prefer', 'manual', 'automatic', 'believe', 'automatic', 'manufacture', 'people', 'physical', 'disability', 'otherwise', 'would', 'able', 'drive', 'automatic_transmission', 'allow', 'driver', 'lazy', 'time', 'available', 'fiddle', 'radio', 'look', 'scenery', 'instead', 'concentrate', 'road', 'manual_transmission', 'keep', 'drive', 'always', 'something', 'grant', 'be', 'not', 'large', 'movement', 'drive', 'fun', 'drive', 'manual', 'fun', 'drive', 'automatic', 'chore', 'case', 'shift', 'speed', 'automatic', 'make', 'shift', 'far', 'faster', 'human', 'could', 'move', 'stick', 'race', 'would', 'want', 'automatic', 'normal', 'driving', 'go', 'manual']]


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 5), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 5), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2)]]


In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('allow', 1),
  ('always', 1),
  ('automatic', 5),
  ('automatic_transmission', 1),
  ('available', 1),
  ('be', 1),
  ('believe', 1),
  ('bigot', 1),
  ('brian', 1),
  ('cary', 1),
  ('case', 1),
  ('chore', 1),
  ('concentrate', 1),
  ('could', 1),
  ('disability', 1),
  ('drive', 5),
  ('driver', 1),
  ('driving', 1),
  ('far', 1),
  ('faster', 1),
  ('fiddle', 1),
  ('fun', 2),
  ('go', 1),
  ('grant', 1),
  ('host', 1),
  ('human', 1),
  ('instead', 1),
  ('institute_troy', 1),
  ('keep', 1),
  ('kens', 1),
  ('large', 1),
  ('lazy', 1),
  ('line', 1),
  ('look', 1),
  ('make', 1),
  ('manual', 3),
  ('manual_shift', 1),
  ('manual_transmission', 1),
  ('manufacture', 1),
  ('move', 1),
  ('movement', 1),
  ('nntp_post', 1),
  ('normal', 1),
  ('not', 1),
  ('organization', 1),
  ('otherwise', 1),
  ('people', 1),
  ('physical', 1),
  ('prefer', 1),
  ('race', 1),
  ('radio', 1),
  ('rensselaer_polytechnic', 1),
  ('reply', 1),
  ('road', 1),
  ('rpi', 1),
  ('sce

In [12]:
def average_precision_at_k(theta, labels, k, dist_fn):
    dist = dist_fn(theta)
    np.fill_diagonal(dist, np.inf)
    idx = np.argpartition(dist, k-1)[:k,:]
    return np.mean(labels[idx] == labels[None,:])

In [13]:
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances

def hellinger(theta):
    return euclidean_distances(np.sqrt(theta))

def dot(theta):
    return 1 - np.dot(theta, theta.transpose())

In [14]:
from numba import jit, prange, autojit

In [15]:
@jit(nopython = True, parallel = True)
def outer(X, fn):
    dist = np.empty((X.shape[0], X.shape[0]), dtype = np.float32)
    for i in prange(X.shape[0]):
        for j in prange(i):
            dist[j,i] = dist[i,j] = fn(X[i,:], X[j,:])           
    return dist

In [16]:
@jit(nopython = True)
def kl(p, q):
    return np.dot(p, np.log(q / p))

In [17]:
@jit(nopython = True)
def sym_kl(p, q):
    return kl(p, q) + kl(q, p)

In [18]:
@jit(nopython = True)
def js(p, q):
    m = (p + q) / 2
    return kl(p, m) / 2 + kl(q, m) / 2

In [19]:
@jit(nopython = True)
def bc(p, q):
    return -np.log(np.sum(np.sqrt(p * q)))

In [20]:
dist_fns = {'cosine' : cosine_distances,
            'eucl' : euclidean_distances,
            'hellinger' : hellinger,
            'dot' : dot,
            'sym_kl' : lambda x: outer(x, sym_kl),
            'js' : lambda x: outer(x, js),
            'bc' : lambda x: outer(x, bc)}

In [21]:
from gensim import matutils

def eval_topic_model(model, labels, k = 100, dist_fns = dist_fns):
    matrix = model.get_document_topics(corpus, minimum_probability=0)
    theta = matutils.corpus2dense(matrix, num_docs=len(corpus), num_terms=lda_model.num_topics, dtype = np.float32)
    theta = np.asarray(theta, dtype = np.float32)
    theta = theta.transpose()
    best_score = 0
    best_dist = None
    for (dist, f) in dist_fns.items():
        score = average_precision_at_k(theta, labels, 100, f)
        if score > best_score:
            best_score, best_dist = score, dist
    return best_score, best_dist

In [26]:
for num_topics in range(5, 60 + 1,5):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    score, dist = eval_topic_model(lda_model, df.target)
    print("%2i topics AP@100 = %.3f (%s distance)" % (num_topics, score, dist))

D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


 5 topics AP@100 = 0.454 (bc distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


10 topics AP@100 = 0.630 (js distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


...

50 topics AP@100 = 0.577 (sym_kl distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


55 topics AP@100 = 0.612 (js distance)


D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
D:\Anaconda\lib\site-packages\sklearn\metrics\pairwise.py:258: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


60 topics AP@100 = 0.690 (sym_kl distance)


In [27]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [28]:
pprint(lda_model.print_topics())

[(0,
  '0.015*"perry_metzger" + 0.008*"clarification" + 0.006*"lui_meme" + '
  '0.006*"laissez_faire" + 0.006*"laissez_passer" + 0.004*"partnership" + '
  '0.000*"bbs" + 0.000*"police" + 0.000*"bis_modem" + 0.000*"intelligble"'),
 (1,
  '0.033*"not" + 0.027*"line" + 0.024*"organization" + 0.023*"would" + '
  '0.022*"write" + 0.019*"do" + 0.017*"article" + 0.017*"get" + 0.016*"be" + '
  '0.013*"know"'),
 (2,
  '0.076*"battery" + 0.051*"temperature" + 0.031*"reaction" + 0.030*"cold" + '
  '0.028*"picture" + 0.025*"cool" + 0.019*"dirt" + 0.017*"warm" + 0.017*"dod" '
  '+ 0.017*"promote"'),
 (3,
  '0.042*"increase" + 0.033*"effect" + 0.033*"patient" + 0.031*"risk" + '
  '0.028*"disease" + 0.027*"food" + 0.026*"medical" + 0.026*"doctor" + '
  '0.023*"cause" + 0.021*"certain"'),
 (4,
  '0.038*"account" + 0.035*"return" + 0.031*"michael" + 0.029*"random" + '
  '0.028*"date" + 0.025*"frank" + 0.024*"pub" + 0.022*"thought" + '
  '0.022*"legislation" + 0.022*"ranger"'),
 (5,
  '0.168*"gun" + 0.0

In [29]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

D:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.370780  0.225753       1        1  49.246956
9     -0.360397  0.116623       2        1  20.200420
17    -0.257665 -0.181023       3        1   7.536002
13    -0.128050 -0.204301       4        1   3.654622
6     -0.099190  0.113366       5        1   3.110790
16    -0.033551 -0.147529       6        1   2.208946
3     -0.024635 -0.119419       7        1   2.156936
18     0.008488 -0.019524       8        1   2.074905
10    -0.064212 -0.148370       9        1   1.915426
14     0.018827  0.008608      10        1   1.473235
12     0.068525  0.053519      11        1   1.282957
5      0.057616  0.036806      12        1   1.187146
4      0.097167  0.009636      13        1   0.995987
7      0.132705  0.032965      14        1   0.866934
19     0.137272  0.029569      15        1   0.567179
2      0.130109  0.040404      16        1   0.554038
15     0.154389  0.037121      17        1   0.449997
11     0.163859  0.037470      18        1   0.381272
8      0.180088  0.038829      19        1   0.110768
0      0.189433  0.039499      20        1   0.025492, topic_info=      Category          Freq                Term         Total  loglift  \
term                                                                      
337    Default   4175.000000                game   4175.000000  30.0000   
44     Default  19598.000000                 not  19598.000000  29.0000   
558    Default   5987.000000                year   5987.000000  28.0000   
145    Default   3107.000000                 key   3107.000000  27.0000   
33     Default  16164.000000                line  16164.000000  26.0000   
1695   Default   2442.000000                 gun   2442.000000  25.0000   
287    Default   2774.000000               space   2774.000000  24.0000   
1610   Default   3108.000000                 car   3108.000000  23.0000   
45     Default  14531.000000        organization  14531.000000  22.0000   
186    Default  13183.000000               write  13183.000000  21.0000   
1512   Default   2465.000000                team   2465.000000  20.0000   
73     Default  11699.000000                  do  11699.000000  19.0000   
973    Default   2748.000000            internet   2748.000000  18.0000   
210    Default   8023.000000                good   8023.000000  17.0000   
906    Default   2061.000000                 law   2061.000000  16.0000   
200    Default  10489.000000             article  10489.000000  15.0000   
1007   Default   3366.000000              system   3366.000000  14.0000   
995    Default   2040.000000                play   2040.000000  13.0000   
63     Default  14116.000000               would  14116.000000  12.0000   
136    Default  10125.000000                 get  10125.000000  11.0000   
47     Default   6097.000000              people   6097.000000  10.0000   
6      Default   9793.000000                  be   9793.000000   9.0000   
705    Default   2614.000000         information   2614.000000   8.0000   
181    Default   5024.000000                 use   5024.000000   7.0000   
708    Default   3625.000000                 new   3625.000000   6.0000   
1492   Default   1775.000000              player   1775.000000   5.0000   
475    Default   1993.000000          government   1993.000000   4.0000   
564    Default   1150.000000                bike   1150.000000   3.0000   
156    Default   6247.000000                 may   6247.000000   2.0000   
23     Default   7564.000000                  go   7564.000000   1.0000   
...        ...           ...                 ...           ...      ...   
26524  Topic20      0.005587         perry_metzg      1.128570   2.9663   
9183   Topic20      0.005587      lehman_brother      1.128570   2.9663   
22187  Topic20      0.005587             vocoded      1.128569   2.9663   
26726  Topic20      0.005587       authorisation      1.128569   2.96

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -15.334646052543322

Coherence Score:  0.452378722696637
